In [14]:
# import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim import models, corpora, similarities
import re
from nltk.stem.porter import PorterStemmer
import time
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim
import os

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
print('Downloads Complete')

Downloads Complete


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xijieguo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xijieguo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/xijieguo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
stop_words = stopwords.words('english')
stemmer = PorterStemmer()

def initial_clean(text):
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower() # lower case the text
    text = nltk.word_tokenize(text)
    return text

def remove_stop_words(text):
    return [word for word in text if word not in stop_words]

def pos(word):
    return nltk.pos_tag([word])[0][1]

informative_pos = ('JJ','VB', 'NN','RBS','VBP','IN','RBR','JJR','JJS','PDT','RP','UH','FW','NNS','VBN','VBG')

def remove_uninformative_pos(text):
    tagged_words = nltk.pos_tag(text)
    return [word for word, tag in tagged_words if tag in informative_pos]
  
clutter = ['food','place','good','order','great','like',
           'service','time','go','ordered','get','love',
           'best','come','eat','dont','tried','try','ask',
           'nice','restaurant','ive','im','didnt']

def remove_garbage(text):
    return [word for word in text if word not in clutter]

def stem_words(text):
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    return stem_words(remove_garbage(remove_uninformative_pos(remove_stop_words(initial_clean(text)))))

def get_top_k_words(df, k = 10000):
    # first get a list of all words
    all_words = [word for item in list(df['tokenized']) for word in item]
    
    # use nltk fdist to get a frequency distribution of all words
    fdist = FreqDist(all_words)
    
    # define a function only to keep words in the top k words
    top_k_words, _ = zip(*fdist.most_common(k))
    top_k_words = set(top_k_words)
    
    return top_k_words

def keep_top_k_words(text, *top_k_words):
    return [word for word in text if word in top_k_words]

def transform_dataset(df):
    # format the columns
    df = df.groupby(['business_id', 'name'])['text'].apply(' '.join).reset_index()
    df = df[df['text'].map(type) == str]
    df.dropna(axis=0, inplace=True, subset=['text'])
    return df

def gen_tokenized_column(df):
    # preprocess the text and business name and create new column "tokenized"
    df['tokenized'] = df['text'].apply(apply_all)
    top_k_words = get_top_k_words(df)
    df['tokenized'] = df['tokenized'].apply(keep_top_k_words, args=(top_k_words))
    return df

def preprocess_dataset(df):
    t1 = time.time()
    preprocessed_df = gen_tokenized_column(transform_dataset(df))
    t2 = time.time()
    print("Time to clean and tokenize", len(df), "businesses' reviews:", (t2-t1)/60, "min")
    return preprocessed_df
    
def get_coherence_score(model, texts, dictionary):
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    return coherence_score

def get_dictionary_corpus(data):
    dictionary = corpora.Dictionary(data)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    return dictionary, corpus

def get_perplexity(model, corpus):
    # a measure of how good the model is; lower the better
    return lda_model.log_perplexity(corpus)

def train_lda(corpus, id2word, chunksize=300, num_topics=12, alpha=1e-2, eta=0.5e-2, passes=1,
              minimum_probability=0.0, random_state=None):
    """
    This function trains the lda model
    We setup parameters like number of topics, the chunksize to use in Hoffman method
    """
    t1 = time.time()
    # low alpha means each document is only represented by a small number of topics, and vice versa
    # low eta means each topic is only represented by a small number of words, and vice versa
    lda = LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=alpha, eta=eta, 
                   chunksize=chunksize, minimum_probability=minimum_probability, passes=passes, 
                   random_state=random_state)
    
    t2 = time.time()
    print("Time to train LDA model on ", len(df), "businesses: ", (t2-t1)/60, "min")
    
    return lda

def get_topic_dist(model, corpus):
    doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in model[corpus]])
    return doc_topic_dist

def train_hdp(random_state=None):
    """
    This function trains the hdp model
    We setup parameters like number of topics, the chunksize to use in Hoffman method
    """
    t1 = time.time()
    # low alpha means each document is only represented by a small number of topics, and vice versa
    # low eta means each topic is only represented by a small number of words, and vice versa
    hdp = HdpModel(corpus=corpus, id2word=dictionary, random_state=random_state)
    t2 = time.time()
    print("Time to train HDP model on ", len(df), "businesses: ", (t2-t1)/60, "min")
    
    return hdp

def train_lsi(num_topics=12, chunksize=300):
    """
    This function trains the lsi model
    We setup parameters like number of topics, the chunksize to use in Hoffman method
    """
    t1 = time.time()
    # low alpha means each document is only represented by a small number of topics, and vice versa
    # low eta means each topic is only represented by a small number of words, and vice versa
    lsi = LsiModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, chunksize=chunksize)
    t2 = time.time()
    print("Time to train LSI model on ", len(df), "businesses: ", (t2-t1)/60, "min")
    
    return lsi

def jensen_shannon(query, matrix):
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix
    m = 0.5*(p + q)
    return np.sqrt(0.5*(entropy(p,m) + entropy(q,m)))
  
def get_most_similar_documents(query,matrix,k=10):
    """
    This function implements the Jensen-Shannon distance above
    and retruns the top k indices of the smallest jensen shannon distances
    """
    sims = jensen_shannon(query,matrix) # list of jensen shannon distances
    return sims.argsort()[:k] # the top k positional index of the smallest Jensen Shannon distances

def get_most_similar_businesses(query_data, model):
    query_bow = dictionary.doc2bow(query_data)
    new_doc_dist = np.array([tup[1] for tup in model[query_bow]])
    most_sim_ids = get_most_similar_documents(new_doc_dist, lda_topic_dist)
    return most_sim_ids


<>:5: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \S
<ipython-input-15-fa6438b61ceb>:5: DeprecationWarning: invalid escape sequence \S
  text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)


In [34]:
csv_name = 'clean_mesa.csv'
df = pd.read_csv(csv_name, index_col=0)
df['tokenized'] = df['tokenized'].apply(eval)

In [37]:
texts = df['tokenized']
dictionary, corpus = get_dictionary_corpus(texts)

In [28]:
# Find optimal number of topics by computing coherence score for LDA
def compute_coherence_LDA(dictionary, corpus, texts, end, start=3, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    end : Max num of topics
    start: Min num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    model_list = []
    coherence_values = []

    for num_topics in range(start, end + 1, step):
        model = train_lda(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=0)
        coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())
        print('progress: num of topics: ', num_topics)
    
    plt.plot(range(start, end + 1, step), coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

    return model_list, coherence_values

models, corherence_values = compute_coherence_LDA(dictionary=dictionary, corpus=corpus, texts=texts, end=30)


In [ ]:
# Find optimal number of topics by computing coherence score for LSI


In [41]:
lda[corpus[0]]

[(1, 0.016919672),
 (2, 0.086762264),
 (3, 0.032220233),
 (4, 0.23659779),
 (5, 0.022509795),
 (8, 0.01265313),
 (9, 0.030311227),
 (10, 0.17654638),
 (11, 0.37557247)]

In [42]:
lda.get_document_topics(corpus[0])

[(1, 0.025580758),
 (2, 0.09432872),
 (3, 0.04854446),
 (4, 0.20399308),
 (5, 0.02056593),
 (8, 0.026410503),
 (9, 0.036607433),
 (10, 0.198711),
 (11, 0.33592972)]

In [8]:
query = pd.read_csv(csv_name, index_col=0)
query = query.loc[query['name']=="Alessia's Ristorante Italiano"]
query = query.groupby(['name'])['text'].apply(' '.join).reset_index()
query = query[query['text'].map(type) == str]
query.dropna(axis=0, inplace=True, subset=['text'])
query['tokenized'] = query['text'].apply(apply_all)

In [0]:
# get the ids of the most similar businesses
new_bow = dictionary.doc2bow(query.iloc[0,2])
new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
most_sim_ids = get_most_similar_documents(new_doc_distribution, doc_topic_dist)

# print the results
most_similar_df = df[df.index.isin(most_sim_ids)]
print('Similar to "{}": \n{}'.format(query['name'][0], most_similar_df['name'].reset_index(drop=True)))

In [0]:
# import pickle

# pickle.dump(doc_topic_dist,open("processor.pkl","wb"))
# preprocessed = pickle.load(open("processor.pkl","rb"))

# most_sim_ids = get_most_similar_documents(new_doc_distribution,preprocessed)

# # print the results
# most_similar_df = df[df.index.isin(most_sim_ids)]
# print('Similar to "{}": \n{}'.format(query['name'][0], most_similar_df['name'].reset_index(drop=True)))

In [0]:
# def process_query(preprocessed, query):
#   # SQL to pandas DataFrame w/ query
#   query = query.groupby(['name'])['text'].apply(' '.join).reset_index()
#   query = query[query['text'].map(type) == str]
#   query.dropna(axis=0, inplace=True, subset=['text'])
#   query['tokenized'] = query['text'].apply(apply_all)
  
#   # read the cached pickle
#   preprocessed = pickle.load(open("processor.pkl","rb"))
  
#   # get the ids of the most similar businesses
#   new_bow = dictionary.doc2bow(query.iloc[0,2])
#   new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
#   most_sim_ids = get_most_similar_documents(new_doc_distribution,preprocessed)

#   # print the results
#   most_similar_df = df[df.index.isin(most_sim_ids)]
#   print('Similar to "{}": \n{}'.format(query['name'][0], most_similar_df['name'].reset_index(drop=True)))

In [0]:
def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    step = 1
    for num_topics in range(60, limit, step):
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
        print('Level {} Complete'.format(num_topics))
        
    # Show graph
    x = range(60, limit, step)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v

In [0]:
%%time
lmlist, c_v = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=df['tokenized'], limit=70)